<a href="https://colab.research.google.com/github/rkrissada/100DayOfMLCode/blob/master/day_061_CLV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("data.csv")
df.head()

UnicodeDecodeError: ignored

In [0]:
import seaborn as sns

sns.set_palette("husl")
sns.set(rc={'image.cmap':'coolwarm'})

%matplotlib inline

In [4]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [0]:
import datetime as dt
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.date

In [0]:
df = df[pd.notnull(df['CustomerID'])]

In [0]:
df = df[(df['Quantity']>0)]

In [0]:
df['Sales'] = df['Quantity'] * df['UnitPrice']
cols_of_interest = ['CustomerID', 'InvoiceDate', 'Sales']
df = df[cols_of_interest]

In [9]:
print(df.head())
print('Number of Entries: %s' % len(df))

   CustomerID InvoiceDate  Sales
0     17850.0  2010-12-01  15.30
1     17850.0  2010-12-01  20.34
2     17850.0  2010-12-01  22.00
3     17850.0  2010-12-01  20.34
4     17850.0  2010-12-01  20.34
Number of Entries: 15691


   * requency represents the number of repeat purchases the customer has made. This means that it’s one less than the total number of purchases.
   * T represents the age of the customer in whatever time units chosen (daily, in our dataset). This is equal to the duration between a customer’s first purchase and the end of the period under study.
   * recency represents the age of the customer when they made their most recent purchases. This is equal to the duration between a customer’s first purchase and their latest purchase. (Thus if they have made only 1 purchase, the recency is 0.)

In [11]:
!pip install lifetimes

    100% |████████████████████████████████| 583kB 6.9MB/s 
    100% |████████████████████████████████| 10.1MB 3.4MB/s 
  Found existing installation: pandas 0.23.4
    Uninstalling pandas-0.23.4:
      Successfully uninstalled pandas-0.23.4


In [0]:
from lifetimes import *

In [11]:
df = df[pd.notnull(df['CustomerID'])]
df.isnull().sum(axis=0)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Check the min and max values in unit prices column

In [12]:
df.UnitPrice.min()

0.03

In [13]:
df.Quantity.min()

-9360.0

Remove the negative values in Quantity column

In [0]:
df = df[(df['Quantity']>0)]

In [15]:
df.Quantity.min()

1.0

After cleaning up, we now dealing with 8224 rows and 8 columns

In [20]:
df.shape

(8224, 8)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8224 entries, 0 to 12460
Data columns (total 8 columns):
InvoiceNo      8224 non-null object
StockCode      8224 non-null object
Description    8224 non-null object
Quantity       8224 non-null float64
InvoiceDate    8224 non-null object
UnitPrice      8224 non-null float64
CustomerID     8224 non-null float64
Country        8224 non-null object
dtypes: float64(3), object(5)
memory usage: 578.2+ KB


Check unique value for each column

In [21]:
def unique_counts(df):
  for i in df.columns:
    count = df[i].nunique()
    print(i, ": ", count)

unique_counts(df)

InvoiceNo :  423
StockCode :  1731
Description :  1737
Quantity :  70
InvoiceDate :  385
UnitPrice :  85
CustomerID :  321
Country :  1


In [24]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom


Add a column for total price

In [25]:
df['TotalProce'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalProce
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


Find out first and last order date in the data

In [26]:
df['InvoiceDate'].min()

'12/1/2010 10:19'

In [27]:
df['InvoiceDate'].max()

'12/6/2010 9:52'

Since recency is calculated for a point in time. The last invoice date is 6/12/2010, this is the date we will use to calculate recency.

In [0]:
import datetime as dt
NOW = dt